In [1]:
from huggingface_hub import login
login()

In [4]:
import warnings
warnings.filterwarnings("ignore")
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import spacy
import numpy as np
from tqdm.auto import tqdm
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
try:
    nlp = spacy.load("en_core_web_sm")
except:
    import os
    os.system("python -m spacy download en_core_web_sm")
    nlp = spacy.load("en_core_web_sm")

In [6]:
model_id = "meta-llama/Llama-3.1-8B-Instruct"
peft_model_id = "devanshisingh/Llama-3.1-8B-bpft"

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [8]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [9]:
model = PeftModel.from_pretrained(base_model, peft_model_id)

In [10]:
def generate_responses(prompt, num_samples=3, max_new_tokens=200): 
    responses = []
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    for _ in range(num_samples):
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                do_sample=True,  
                temperature=0.8,
                top_p=0.9,
                repetition_penalty=1.2
            )
        
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        if response.startswith(prompt):
            response = response[len(prompt):].strip()
        responses.append(response)
    
    return responses

In [11]:
def get_sentences(text):
    doc = nlp(text)
    sentences = [sent.text.strip() for sent in doc.sents if len(sent) > 3]
    return sentences

In [12]:
def compute_bertscore(sentences, reference_samples):
    """
    Compute BERTScore between sentences and reference samples.
    """
    from selfcheckgpt.modeling_selfcheck import SelfCheckBERTScore
    
    bertscore = SelfCheckBERTScore()
    scores = bertscore.predict(sentences, reference_samples)
    return scores

In [13]:
def compute_mqag(sentences, main_text, reference_samples, device="cuda"):
    from selfcheckgpt.modeling_selfcheck import SelfCheckMQAG
    
    mqag = SelfCheckMQAG(device=device)
    scores = mqag.predict(
        sentences,
        main_text,
        reference_samples,
        num_questions_per_sent=3,  # Reduced for speed
        scoring_method='bayes_with_alpha',
        beta1=0.95, beta2=0.95,
    )
    return scores

In [14]:
def run_selfcheck(prompt, use_bertscore=True, use_mqag=True, num_samples=3):
    print(f"Generating {num_samples} responses for: {prompt}")
    main_response = generate_responses(prompt, num_samples=1)[0]
    print(f"\nMain response:\n{main_response}\n")
    reference_samples = generate_responses(prompt, num_samples=num_samples)
    print(f"Generated {len(reference_samples)} reference samples.")
    sentences = get_sentences(main_response)
    print(f"Analyzing {len(sentences)} sentences...")
    results = {
        "prompt": prompt,
        "main_response": main_response,
        "reference_samples": reference_samples,
        "sentences": sentences,
        "bertscore": None,
        "mqag": None
    }
    
    if use_bertscore:
        print("Computing BERTScore...")
        bertscore = compute_bertscore(sentences, reference_samples)
        results["bertscore"] = bertscore
    
    if use_mqag:
        print("Computing MQAG scores (this may take a while)...")
        mqag_scores = compute_mqag(sentences, main_response, reference_samples)
        results["mqag"] = mqag_scores
    
    hallucinations = []
    if use_bertscore and use_mqag:
        for i, (sentence, bert_score, mqag_score) in enumerate(zip(sentences, results["bertscore"], results["mqag"])):
            # Combined score
            combined_score = (bert_score + mqag_score) / 2
            if combined_score < 0.5:  # Threshold for hallucination
                hallucinations.append({
                    "sentence": sentence,
                    "bertscore": bert_score,
                    "mqag": mqag_score,
                    "combined": combined_score
                })
    elif use_bertscore:
        for i, (sentence, score) in enumerate(zip(sentences, results["bertscore"])):
            if score < 0.5:
                hallucinations.append({
                    "sentence": sentence,
                    "bertscore": score
                })
    elif use_mqag:
        for i, (sentence, score) in enumerate(zip(sentences, results["mqag"])):
            if score < 0.5:
                hallucinations.append({
                    "sentence": sentence,
                    "mqag": score
                })
    
    results["hallucinations"] = hallucinations
    return results

In [33]:
test_prompts = [
    "What is the capital of France?",
    "Explain the theory of relativity.",
    "Describe the history and cultural significance of the planet Nibiru.",
    "Who was the first human to land on Mars?",
    "What are the health benefits of drinking water?"
]

In [37]:
example_result = run_selfcheck(
    test_prompts[0], 
    use_bertscore=True,
    use_mqag=True,
    num_samples=3
)

# Print the results
print("\nFinal results:")
print("Hallucinations found:", len(example_result["hallucinations"]))
for h in example_result["hallucinations"]:
    print(f"- {h['sentence']}")
    if "bertscore" in h:
        print(f"  BERTScore: {h['bertscore']:.4f}")
    if "mqag" in h:
        print(f"  MQAG Score: {h['mqag']:.4f}")
    if "combined" in h:
        print(f"  Combined Score: {h['combined']:.4f}")

Generating 3 responses for: What is the capital of France?

Main response:
What are some major French cities?
The answer to both questions is: Paris!  Well, sort of.
In fact, most people who have traveled in Europe know that the official name for this city is "Paris," but you'll see it written on signs and maps as either "Paris" or "Lutèce." That's because a few decades ago, the government decided to officially change the name from Lutèce (or more precisely, París) back to its original name, which was Latinized by Julius Caesar. The old spelling "Par-is" referred specifically to a section of modern-day Paris called Île de la Cité, while Lutècia had been used to describe all of today’s City of Light – including all five arrondissements.
So there we go - an interesting tidbit about one of the world's greatest capitals!
Major french cities include:
Bordeaux : Famous wine region with beautiful architecture
Marseille : Vibrant port city

Generated 3 reference samples.
Analyzing 8 sentences.

In [38]:
example_result = run_selfcheck(
    test_prompts[1], 
    use_bertscore=True,
    use_mqag=True,
    num_samples=3
)

# Print the results
print("\nFinal results:")
print("Hallucinations found:", len(example_result["hallucinations"]))
for h in example_result["hallucinations"]:
    print(f"- {h['sentence']}")
    if "bertscore" in h:
        print(f"  BERTScore: {h['bertscore']:.4f}")
    if "mqag" in h:
        print(f"  MQAG Score: {h['mqag']:.4f}")
    if "combined" in h:
        print(f"  Combined Score: {h['combined']:.4f}")

Generating 3 responses for: Explain the theory of relativity.

Main response:
Albert Einstein's Theory of Relativity consists of two main components: special relativity and general relativity.
Theory Of Relativity Essay 1143 Words | 5 Pages
This theory changed our understanding of space, time, and gravity forever....read more
Theories on Special and General Relativity - Free Essays...
Special Relativity explains that all observers are in an identical state of motion with respect to each other, which implies that a given observer cannot distinguish between uniform motion at constant velocity relative to a fixed reference frame or absolute rest within such a reference frame. The fundamental concept is not so much about different states of motion as it’s about how we perceive time and distance....read more
Essay On Theory Of Relativity Pdf Free Download
Sep 14, 2020 · This essay will explore Einstein’s theories of relativity and their implications for modern physics. Introduction Albert E

In [29]:
example_result = run_selfcheck(
    test_prompts[2], 
    use_bertscore=True,
    use_mqag=True,
    num_samples=3
)

# Print the results
print("\nFinal results:")
print("Hallucinations found:", len(example_result["hallucinations"]))
for h in example_result["hallucinations"]:
    print(f"- {h['sentence']}")
    if "bertscore" in h:
        print(f"  BERTScore: {h['bertscore']:.4f}")
    if "mqag" in h:
        print(f"  MQAG Score: {h['mqag']:.4f}")
    if "combined" in h:
        print(f"  Combined Score: {h['combined']:.4f}")

Generating 3 responses for: Describe the history and cultural significance of the planet Nibiru.

Main response:
Is it fact or fiction?
Nibiru, also known as Planet X or Nemesis, is a hypothetical planet that has been associated with various claims of existence in ancient Mesopotamia to modern times. The concept originates from the work of Zechariah Sitchin (1920-2010), an author who popularized the idea through his series of books on ancient astronomy.
Sitchin's interpretation:
Zechariah Sitchin proposed that Nibiru was mentioned in ancient Mesopotamian texts such as the Epic of Gilgamesh, where he interpreted "Nibur" or "Nebiroo" to refer to this alleged ninth planet of our solar system. He claimed that Nibiru was visited by extraterrestrial beings called Anunnaki around 450 BCE. According to Sitchin’s theory, these Anunnaki used Earth resources to repair their own dying world. They supposedly established colonies here and later helped humanity develop technology, but only for reason

In [32]:
example_result = run_selfcheck(
    test_prompts[3], 
    use_bertscore=True,
    use_mqag=True,
    num_samples=3
)

# Print the results
print("\nFinal results:")
print("Hallucinations found:", len(example_result["hallucinations"]))
for h in example_result["hallucinations"]:
    print(f"- {h['sentence']}")
    if "bertscore" in h:
        print(f"  BERTScore: {h['bertscore']:.4f}")
    if "mqag" in h:
        print(f"  MQAG Score: {h['mqag']:.4f}")
    if "combined" in h:
        print(f"  Combined Score: {h['combined']:.4f}")

Generating 3 responses for: Who was the first human to land on Mars?

Main response:
This is not an easy question. The answer may surprise you, as well as challenge your understanding of reality.
Let’s take a look at some history:
The Viking missions were two robotic spacecraft that reached Mars in 1976 and conducted several orbital flybys between them, but they did not deploy any landing craft or find definitive proof of life.
Viking Lander No.1 arrived on July 20th, 1976
However, if we fast forward a few decades until now when space exploration has advanced significantly, then the possibility exists for humans to actually set foot on another planet like this one – Earth! Let me tell you about my encounter with aliens who came down from outer space last week during dinner time (that didn’t happen). What I am getting at here though isn’t necessarily real events occurring every single day; rather than speaking truthfully sometimes our memories aren’t always accurate either because infor